In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("Libraries imported successfully!")

Libraries imported successfully!


## data preprocessing - Cleaning

In [2]:
# Load the dataset
data_path = '../data/exl_credit_card_churn_data.csv'
df = pd.read_csv(data_path)

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")

# Display first few rows
df.head()

Dataset loaded successfully!
Shape: (1010, 10)


,CustomerID,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Churn
0,CUST0001,Male,56.0,4.0,0.00,4.0,0.0,0.0,40282.42,1.0
1,CUST0002,NaN,28.0,8.0,67408.01,4.0,0.0,1.0,27333.51,0.0
2,CUST0003,Female,47.0,6.0,1154.97,1.0,0.0,1.0,99514.91,1.0
3,CUST0004,Male,42.0,1.0,0.00,2.0,1.0,1.0,146588.22,0.0
4,CUST0005,Male,64.0,3.0,77109.94,4.0,0.0,0.0,131792.25,0.0


In [3]:
df['Churn'].isna().sum()

np.int64(3)

In [4]:
df['Churn'].value_counts()

Churn
0.0      523
1.0      478
2          3
2.0        2
Maybe      1
Name: count, dtype: int64

In [5]:
# remove the CustomerID column (not needed for analysis)
df.drop(columns=['CustomerID'], inplace=True)

In [6]:
# Check for missing values
missing_values = df.isnull().sum()
missing_values

Gender             6
Age                2
Tenure             3
Balance            4
NumOfProducts      4
HasCrCard          2
IsActiveMember     5
EstimatedSalary    1
Churn              3
dtype: int64

In [7]:
# fill the null values
# for gender,Churn,hascrcard,isactivemember -> fill mode
# for age, Tenure,balance,numofproducts,estimatedsalary -> fill mean

df['Gender'] = df['Gender'].fillna(df['Gender'].mode()[0])
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['Tenure'] = df['Tenure'].fillna(df['Tenure'].mean())
df['Balance'] = df['Balance'].fillna(df['Balance'].mean())
df['NumOfProducts'] = df['NumOfProducts'].fillna(df['NumOfProducts'].mean())
df['EstimatedSalary'] = df['EstimatedSalary'].fillna(df['EstimatedSalary'].mean())
df['Churn'] = df['Churn'].fillna(df['Churn'].mode()[0])
df['HasCrCard'] = df['HasCrCard'].fillna(df['HasCrCard'].mode()[0])
df['IsActiveMember'] = df['IsActiveMember'].fillna(df['IsActiveMember'].mode()[0])

# Check for null values again
null_values = df.isnull().sum()
print("\nNull values after filling:")
print(null_values)


Null values after filling:
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Churn              0
dtype: int64


In [8]:
df['Churn'].value_counts()

Churn
0.0      526
1.0      478
2          3
2.0        2
Maybe      1
Name: count, dtype: int64

In [9]:
from numpy import dtype

columns = df.dtypes.values
dtypes = df.dtypes.index

print("Before fixing object columns:")
for c,d in zip(columns, dtypes):
    # print unique of each object column
    if c == dtype('O') and d not in ['CustomerID']:
        print(f"Column: {d}, Unique Values: {df[d].unique()}")

print("\n")
# Fix columns Gender, HasCrCard, IsActiveMember and Churn
# for Gender = Use title case
# for HasCrCard = Convert to int and slice to either 0 or 1 (if smthing is > 1 and 0 if smthing is < 0)
# for IsActiveMember = Convert to int and slice to either 0 or 1 (if smthing is > 1 and 0 if smthing is < 0)
# for Churn = Convert to int and slice to either 0 or 1 (if smthing is > 1 and 0 if smthing is < 0) and map 'Maybe' to 2 (for now).

df['Gender'] = df['Gender'].str.strip().str.title()
df['HasCrCard'] = df['HasCrCard'].apply(lambda x: 1 if pd.notna(x) and str(x).isdigit() and float(x) > 0 else 0)
df['IsActiveMember'] = df['IsActiveMember'].apply(lambda x: 1 if pd.notna(x) and str(x).isdigit() and float(x) > 0 else 0)
df['Churn'] = df['Churn'].map({'0.0': 0, '1.0': 1, '2.0': 2, '2': 2, 'Maybe': 2})

columns = df.dtypes.values
dtypes = df.dtypes.index
print("After fixing object columns:")
for c,d in zip(columns, dtypes):
    # print unique of each object column
    if c == dtype('O') and d not in ['CustomerID']:
        print(f"Column: {d}, Unique Values: {df[d].unique()}")

Before fixing object columns:
Column: Gender, Unique Values: ['Male' 'Female' 'FEMALE' ' male ' 'MALE' ' Female']
Column: HasCrCard, Unique Values: ['0.0' '1.0' '2.0' 'Yes']
Column: IsActiveMember, Unique Values: ['0.0' '1.0' '-1' 'No' '-1.0']
Column: Churn, Unique Values: ['1.0' '0.0' 'Maybe' '2.0' '2']


After fixing object columns:
Column: Gender, Unique Values: ['Male' 'Female']


In [10]:
df['Churn'].value_counts()
# Since 2 has very less count, we can drop it for now
df = df[df['Churn'] != 2]

df['Churn'].value_counts()

Churn
0    526
1    478
Name: count, dtype: int64

In [11]:
# save the cleaned dataset
df.to_parquet('../data/exl_credit_card_churn_data_cleaned.parquet', index=False)